<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB42_Mono_vs_Multi_Agent_Interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB42 : Mono Agent versus Multi Agent Interactions

**Objectif:** Expérimenter comment plusieurs agents légers coordonnent (ou entrent en conflit) par rapport à un agent unique accomplissant la même tâche.

**Durée estimée:** 15–20 minutes

**Livrable:** Un notebook qui exécute la même tâche en modes (A) single-agent et (B) multi-agent et compare les résultats.

---

## Architecture

- **Blackboard** (mémoire partagée)
- **Rôles** spécialisés (Researcher, Planner, Critic)
- **Protocole** de messages structuré
- **Comparaison** single-agent vs multi-agent

## 1) Setup :

Installation des dépendances nécessaires

In [26]:
# Installation des packages
!pip install -q openai python-dotenv

### Configuration de la clé API OpenAI

⚠️ **Important:** Remplacez `your_api_key_here` par une clé API OpenAI valide.

Dans Google Colab, vous pouvez aussi utiliser les **Secrets** :
1. Cliquez sur l'icône 🔑 dans la barre latérale gauche
2. Ajoutez un secret nommé `OPENAI_API_KEY`
3. Décommentez la méthode alternative ci-dessous

In [27]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## 2) Définition d'une interface minimale pour agent :

Chaque agent possède :
- **role** : son rôle spécifique
- **prompt** : instructions système
- **act(state)** : méthode retournant un message et des mises à jour optionnelles du blackboard

In [28]:
import os
import uuid
import time
from dataclasses import dataclass, field
from typing import Dict, Any, List
from openai import OpenAI

# Initialisation du client OpenAI
llm = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def chat(system: str, user: str, temp: float = 0) -> str:
    """
    Fonction helper pour appeler l'API OpenAI.

    Args:
        system: Prompt système définissant le rôle
        user: Message utilisateur avec le contexte
        temp: Température (0 = déterministe, >0 = créatif)

    Returns:
        Réponse du modèle
    """
    r = llm.chat.completions.create(
        model="gpt-4o-mini",
        temperature=temp,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ]
    )
    return r.choices[0].message.content.strip()

@dataclass
class Message:
    """Message échangé entre agents."""
    sender: str
    content: str
    ts: float = field(default_factory=time.time)

@dataclass
class Blackboard:
    """
    Blackboard (tableau noir) : mémoire partagée entre agents.
    - data: dictionnaire de données structurées
    - log: historique des messages
    """
    data: Dict[str, Any] = field(default_factory=dict)
    log: List[Message] = field(default_factory=list)

    def write(self, msg: Message):
        """Ajoute un message au log."""
        self.log.append(msg)

    def update(self, **kwargs):
        """Met à jour les données du blackboard."""
        self.data.update(kwargs)

print("✓ Structures de base définies (Message, Blackboard, fonction chat)")

✓ Structures de base définies (Message, Blackboard, fonction chat)


## 3) Création des prompts définissant les rôles de chaque agent :  

Définition des prompts pour chaque rôle spécialisé :
- **Researcher** : collecte des faits et sources
- **Planner** : propose un plan structuré
- **Critic** : identifie les lacunes et améliore
- **Single Agent** : combine tous les rôles

In [29]:
ROLE_RESEARCHER = (
    "Role: Researcher. Read the task & propose 3 concise facts or sources "
    "to inform a solution. Output as bullet points. No final answer."
)

ROLE_PLANNER = (
    "Role: Planner. Read the latest facts and propose an ordered plan with 3–5 steps. "
    "State assumptions and risks briefly."
)

ROLE_CRITIC = (
    "Role: Critic. Review the plan for gaps, conflicts, or missing data. "
    "Return 2–3 actionable improvements. No final answer."
)

ROLE_SINGLE = (
    "You are a single agent acting as Researcher, Planner, and Critic at once. "
    "Given the task, produce facts, a plan, then self-critique with improvements."
)

print("✓ Prompts de rôles définis")
print(f"  - Researcher: {len(ROLE_RESEARCHER)} caractères")
print(f"  - Planner: {len(ROLE_PLANNER)} caractères")
print(f"  - Critic: {len(ROLE_CRITIC)} caractères")
print(f"  - Single: {len(ROLE_SINGLE)} caractères")

✓ Prompts de rôles définis
  - Researcher: 132 caractères
  - Planner: 117 caractères
  - Critic: 120 caractères
  - Single: 150 caractères


## 4) Définition des classes des agents :

Classe `BaseAgent` implémentant le comportement de base d'un agent :
- Lecture du contexte (task + blackboard + historique)
- Appel au LLM avec son prompt de rôle
- Retour d'un message structuré

In [30]:
class BaseAgent:
    """
    Agent de base avec un rôle et un nom.
    """
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role

    def act(self, task: str, bb: Blackboard) -> Message:
        """
        Exécute une action basée sur le task et l'état du blackboard.

        Args:
            task: Tâche à accomplir
            bb: Blackboard contenant l'état partagé

        Returns:
            Message contenant la réponse de l'agent
        """
        # Construction du contexte
        context = (
            f"Task:\n{task}\n\nBlackboard:\n{bb.data}\n\n"
            f"Conversation so far:\n" +
            "\n".join([
                f"- {m.sender}: {m.content[:200]}"
                for m in bb.log[-6:]  # Garde les 6 derniers messages
            ])
        )

        # Appel au LLM
        out = chat(system=self.role, user=context)

        return Message(sender=self.name, content=out)

# Instanciation des agents spécialisés
researcher = BaseAgent("Researcher", ROLE_RESEARCHER)
planner    = BaseAgent("Planner", ROLE_PLANNER)
critic     = BaseAgent("Critic", ROLE_CRITIC)
single     = BaseAgent("Solo", ROLE_SINGLE)

print("✓ Agents instanciés :")
print(f"  - {researcher.name}")
print(f"  - {planner.name}")
print(f"  - {critic.name}")
print(f"  - {single.name} (single-agent)")

✓ Agents instanciés :
  - Researcher
  - Planner
  - Critic
  - Solo (single-agent)


## 5) Boucle d'intéractions multi-agents:

**Protocole:** Researcher → Planner → Critic → Planner (révision)

Après chaque tour :
1. Écriture dans `blackboard.log`
2. Stockage de l'état compact
3. Résumé des messages récents pour le contexte

In [31]:
def summarize_last(msgs: List[Message], role: str) -> str:
    """
    Résume les derniers messages pour un rôle spécifique.
    Utilise le LLM pour condenser l'information.

    Args:
        msgs: Liste des messages
        role: Rôle pour lequel on résume

    Returns:
        Résumé condensé
    """
    if not msgs:
        return ""

    joined = "\n".join([m.content for m in msgs[-2:]])
    return chat(system=f"Summarize for {role}.", user=joined)

def multi_agent_run(task: str, rounds: int = 1) -> Blackboard:
    """
    Exécute le workflow multi-agent.

    Args:
        task: Tâche à accomplir
        rounds: Nombre d'itérations du cycle complet

    Returns:
        Blackboard final avec tous les résultats
    """
    bb = Blackboard(data={
        "task": task,
        "facts": [],
        "plan": "",
        "critique": ""
    })

    for r in range(rounds):
        print(f"\n🔄 Round {r+1}/{rounds}")

        # 1) Researcher collecte des faits
        print("  📚 Researcher: collecte des faits...")
        m1 = researcher.act(task, bb)
        bb.write(m1)
        bb.update(facts=bb.data.get("facts", []) + [m1.content])

        # 2) Planner propose un plan initial
        print("  📋 Planner: création du plan...")
        ctx = summarize_last(bb.log, "Planner")
        m2 = planner.act(task + "\n\nContext:\n" + ctx, bb)
        bb.write(m2)
        bb.update(plan=m2.content)

        # 3) Critic review le plan
        print("  🔍 Critic: analyse et critique...")
        ctx = summarize_last(bb.log, "Critic")
        m3 = critic.act(task + "\n\nContext:\n" + ctx, bb)
        bb.write(m3)
        bb.update(critique=m3.content)

        # 4) Planner révise le plan selon la critique
        print("  ✏️  Planner: révision du plan...")
        ctx = summarize_last(bb.log, "Planner")
        m4 = planner.act(
            task + "\n\nRevise plan per critique:\n" + ctx, bb
        )
        bb.write(m4)
        bb.update(plan=m4.content, round=r+1)

    print("\n✓ Multi-agent workflow terminé")
    return bb

print("✓ Fonction multi_agent_run définie")

✓ Fonction multi_agent_run définie


## 6) Traitement mono/single-agent :

L'agent unique combine tous les rôles en une seule invocation.

In [32]:
def single_agent_run(task: str) -> Blackboard:
    """
    Exécute la tâche avec un seul agent généraliste.

    Args:
        task: Tâche à accomplir

    Returns:
        Blackboard avec le résultat final
    """
    bb = Blackboard(data={"task": task})

    print("\n🤖 Single-agent: traitement...")
    m = single.act(task, bb)
    bb.write(m)
    bb.update(final=m.content)

    print("✓ Single-agent terminé")
    return bb

print("✓ Fonction single_agent_run définie")

✓ Fonction single_agent_run définie


## 7) Lancement des deux modes dans la même tâche de test :

Tâche de test : conception d'une séquence d'emails B2B pour un outil d'IA ciblant les responsables d'analytique dans la santé.

In [33]:
# Définition de la tâche
# TASK = (
#     "Design a 3-email outreach sequence for a B2B AI tool targeting "
#     "healthcare analytics leaders. Include subject lines and a clear CTA each."
# )

TASK = (
    "Conçois une séquence de 3 e-mails pour un outil IA de B2B ciblant les responsables de l'analyse des données du secteur de la santé."
    "Inclus des objets et des appels à l'action clairs dans chaque e-mail."
)

print("="*70)
print("TÂCHE À ACCOMPLIR")
print("="*70)
print(TASK)
print("="*70)

TÂCHE À ACCOMPLIR
Conçois une séquence de 3 e-mails pour un outil IA de B2B ciblant les responsables de l'analyse des données du secteur de la santé.Inclus des objets et des appels à l'action clairs dans chaque e-mail.


In [34]:
# Exécution du mode MULTI-AGENT
print("\n" + "="*70)
print("EXÉCUTION MODE MULTI-AGENT")
print("="*70)

bb_multi = multi_agent_run(TASK, rounds=1)


EXÉCUTION MODE MULTI-AGENT

🔄 Round 1/1
  📚 Researcher: collecte des faits...
  📋 Planner: création du plan...
  🔍 Critic: analyse et critique...
  ✏️  Planner: révision du plan...

✓ Multi-agent workflow terminé


In [35]:
# Exécution du mode SINGLE-AGENT
print("\n" + "="*70)
print("EXÉCUTION MODE SINGLE-AGENT")
print("="*70)

bb_single = single_agent_run(TASK)


EXÉCUTION MODE SINGLE-AGENT

🤖 Single-agent: traitement...
✓ Single-agent terminé


### Affichage des résultats

In [36]:
print("\n" + "="*70)
print("RÉSULTATS MULTI-AGENT")
print("="*70)

print("\n📊 FAITS COLLECTÉS (Researcher):")
print("-" * 70)
for i, fact in enumerate(bb_multi.data.get("facts", []), 1):
    print(f"\nFact {i}:\n{fact}")

print("\n" + "="*70)
print("📋 PLAN FINAL (Planner):")
print("="*70)
print(bb_multi.data.get("plan", ""))

print("\n" + "="*70)
print("🔍 CRITIQUE (Critic):")
print("="*70)
print(bb_multi.data.get("critique", ""))


RÉSULTATS MULTI-AGENT

📊 FAITS COLLECTÉS (Researcher):
----------------------------------------------------------------------

Fact 1:
- **Croissance du marché de l'IA dans la santé** : Selon un rapport de MarketsandMarkets, le marché de l'IA dans le secteur de la santé devrait atteindre 36,1 milliards de dollars d'ici 2025, ce qui souligne l'importance croissante de l'IA pour les responsables de l'analyse des données.

- **Amélioration des résultats cliniques** : Une étude publiée dans le Journal of Medical Internet Research a montré que l'utilisation d'outils d'IA pour l'analyse des données peut améliorer les résultats cliniques de 30%, ce qui est un argument fort pour les responsables de l'analyse des données.

- **Cas d'utilisation réussis** : Des entreprises comme IBM Watson Health et Google Health ont démontré des cas d'utilisation réussis de l'IA pour l'analyse des données de santé, ce qui peut servir d'inspiration pour les responsables de l'analyse des données dans le secteur.

In [37]:
print("\n" + "="*70)
print("RÉSULTATS SINGLE-AGENT")
print("="*70)
print(bb_single.data.get("final", ""))


RÉSULTATS SINGLE-AGENT
### Facts:
1. **Cible**: Responsables de l'analyse des données dans le secteur de la santé.
2. **Outil**: Outil d'intelligence artificielle (IA) B2B.
3. **Objectif**: Présenter l'outil, démontrer sa valeur ajoutée, et inciter à l'action (essai, démo, etc.).

### Plan:
**E-mail 1: Introduction à l'outil IA**
- **Objet**: Transformez vos données de santé en insights exploitables
- **Contenu**: Présentation de l'outil IA, ses fonctionnalités principales, et comment il peut aider à améliorer l'analyse des données dans le secteur de la santé.
- **Appel à l'action**: "Découvrez comment notre outil peut transformer votre analyse des données. Réservez une démo gratuite dès aujourd'hui !"

**E-mail 2: Étude de cas et témoignages**
- **Objet**: Comment [Nom d'une entreprise] a optimisé son analyse des données de santé
- **Contenu**: Présentation d'une étude de cas d'une entreprise ayant utilisé l'outil avec succès, incluant des statistiques et des témoignages.
- **Appel à

## 8) Comparaison des 2 approches :

Critères d'évaluation :
- **Exhaustivité** : Le plan inclut-il les "pain" points, propositiosn de valeurs, CTAs ?
- **Qualité** : Clarté, ton, séquençage
- **Diversité** : Les idées sont-elles moins répétitives en mode multi-agent ?
- **Apport de l'itération** : La critique a-t-elle matériellement amélioré le plan ?

In [38]:
# Évaluation automatique par le LLM
comparison_prompt = f"""
Compare two outputs for the same task.

[Multi-agent Plan]
{bb_multi.data.get('plan', '')}

[Single-agent Output]
{bb_single.data.get('final', '')}

Score each (1–5) on Coverage, Quality, Diversity, and provide a 3-sentence verdict.
"""

print("\n" + "="*70)
print("ÉVALUATION AUTOMATIQUE")
print("="*70)

evaluation = chat("You are a fair evaluator and give the results in french.", comparison_prompt)
print(evaluation)


ÉVALUATION AUTOMATIQUE
### Évaluation des Sorties

#### [Multi-agent Plan]
- **Couverture : 5**
  Le plan couvre de manière exhaustive les étapes nécessaires pour engager les destinataires, avec des e-mails bien structurés et des appels à l'action clairs.

- **Qualité : 5**
  La qualité du contenu est élevée, avec des messages clairs et convaincants qui mettent en avant les avantages de l'outil d'IA.

- **Diversité : 4**
  Bien que le plan présente une bonne variété d'approches (introduction, cas d'utilisation, offre spéciale), il pourrait bénéficier d'une plus grande diversité dans les types de contenu (par exemple, vidéos ou infographies).

#### [Single-agent Output]
- **Couverture : 4**
  Le plan aborde les points essentiels, mais il manque des détails sur les cas d'utilisation et les témoignages, ce qui pourrait renforcer l'impact.

- **Qualité : 4**
  La qualité est bonne, mais le contenu pourrait être amélioré par des éléments visuels et une personnalisation accrue pour capter l

### Statistiques de conversation

In [39]:
print("\n" + "="*70)
print("STATISTIQUES")
print("="*70)

print(f"\n📊 Multi-agent:")
print(f"  - Nombre de messages échangés: {len(bb_multi.log)}")
print(f"  - Agents impliqués: {len(set(m.sender for m in bb_multi.log))}")
print(f"  - Rounds complétés: {bb_multi.data.get('round', 0)}")

print(f"\n🤖 Single-agent:")
print(f"  - Nombre de messages: {len(bb_single.log)}")
print(f"  - Agents impliqués: {len(set(m.sender for m in bb_single.log))}")

print(f"\n📝 Longueur des sorties:")
multi_length = len(bb_multi.data.get('plan', ''))
single_length = len(bb_single.data.get('final', ''))
print(f"  - Multi-agent plan: {multi_length} caractères")
print(f"  - Single-agent output: {single_length} caractères")
print(f"  - Différence: {abs(multi_length - single_length)} caractères")


STATISTIQUES

📊 Multi-agent:
  - Nombre de messages échangés: 4
  - Agents impliqués: 3
  - Rounds complétés: 1

🤖 Single-agent:
  - Nombre de messages: 1
  - Agents impliqués: 1

📝 Longueur des sorties:
  - Multi-agent plan: 3735 caractères
  - Single-agent output: 2608 caractères
  - Différence: 1127 caractères


## 9) Expérimentations & extensions (optionnel)

### Idées d'extensions

1. **Coordinateur** : Ajouter un agent Coordinator qui assigne les sous-tâches et arrête les boucles sur convergence
2. **Mémoire de rôle** : Chaque agent garde ses propres notes à travers les exécutions
3. **Échecs de communication** : Simuler la perte d'un message et observer la robustesse
4. **Seconde tâche** : Tester avec "Create a 5-step incident response checklist for an LLM outage"
5. **Tracking coût/latence** : Mesurer le trade-off entre single et multi-agent

In [40]:
# Exemple d'extension 1: Seconde tâche
# TASK_2 = "Create a 5-step incident response checklist for an LLM outage."
TASK_2 = "Créez une liste de contrôle en 5 étapes pour la réponse aux incidents en cas de panne de LLM."

print("\n" + "="*70)
print("EXPÉRIMENTATION: TÂCHE ALTERNATIVE")
print("="*70)
print(f"Tâche: {TASK_2}")
print("\nDécommentez les lignes ci-dessous pour exécuter:")
print("="*70)

# Décommentez pour exécuter:
# bb_multi_2 = multi_agent_run(TASK_2, rounds=1)
# bb_single_2 = single_agent_run(TASK_2)
# print("\nMulti-agent plan:")
# print(bb_multi_2.data.get("plan", ""))
# print("\nSingle-agent output:")
# print(bb_single_2.data.get("final", ""))


EXPÉRIMENTATION: TÂCHE ALTERNATIVE
Tâche: Créez une liste de contrôle en 5 étapes pour la réponse aux incidents en cas de panne de LLM.

Décommentez les lignes ci-dessous pour exécuter:


In [41]:
# Exemple d'extension 2: Tracking de coût et latence
import time

def timed_run(func, *args, **kwargs):
    """Mesure le temps d'exécution d'une fonction."""
    start = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - start
    return result, elapsed

print("\n" + "="*70)
print("EXPÉRIMENTATION: MESURE DE PERFORMANCE")
print("="*70)
print("Décommentez pour exécuter des benchmarks:")
print("="*70)

# Décommentez pour benchmarker:
# bb_multi_timed, time_multi = timed_run(multi_agent_run, TASK, rounds=1)
# bb_single_timed, time_single = timed_run(single_agent_run, TASK)
#
# print(f"\n⏱️  Temps d'exécution:")
# print(f"  - Multi-agent: {time_multi:.2f}s")
# print(f"  - Single-agent: {time_single:.2f}s")
# print(f"  - Overhead multi-agent: {time_multi - time_single:.2f}s ({((time_multi/time_single - 1) * 100):.1f}%)")
#
# # Estimation du coût (basé sur gpt-4o-mini pricing)
# # Input: $0.150 / 1M tokens, Output: $0.600 / 1M tokens
# print(f"\n💰 Nombre d'appels LLM:")
# print(f"  - Multi-agent: ~{len(bb_multi_timed.log)} appels")
# print(f"  - Single-agent: ~{len(bb_single_timed.log)} appel(s)")


EXPÉRIMENTATION: MESURE DE PERFORMANCE
Décommentez pour exécuter des benchmarks:


## Conclusion

### Observations attendues

**Mode Multi-Agent:**
- ✅ Spécialisation des rôles (recherche, planification, critique)
- ✅ Itération et amélioration progressive
- ✅ Diversité des perspectives
- ⚠️ Plus d'appels LLM (coût et latence)
- ⚠️ Coordination nécessaire

**Mode Single-Agent:**
- ✅ Rapidité (1 seul appel LLM)
- ✅ Cohérence interne
- ✅ Simplicité
- ⚠️ Moins de profondeur dans l'analyse
- ⚠️ Pas d'itération critique

### Quand utiliser chaque approche ?

**Multi-Agent:**
- Tâches complexes nécessitant expertise multiple
- Besoin d'itération et de critique
- Problèmes bénéficiant de perspectives diverses
- Workflows longs où le coût est justifié

**Single-Agent:**
- Tâches simples et bien définies
- Contraintes de temps/coût strictes
- Besoin de réponse rapide
- Problèmes ne nécessitant pas de spécialisation

---

### 📚 Ressources additionnelles

- [LangGraph Documentation](https://python.langchain.com/docs/langgraph)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [Multi-Agent Systems Papers](https://arxiv.org/list/cs.MA/recent)